In [1]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
   r"D:\FinalProject\splitdataset48x48\train",
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    r'D:\FinalProject\splitdataset48x48\val',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)
class_names = list(train_generator.class_indices.keys())
print(class_names)


Found 4090 images belonging to 27 classes.
Found 1031 images belonging to 27 classes.
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'blank']


In [3]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(27, activation='softmax'))

model.summary()

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 128)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 256)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

In [4]:
import os
from keras.callbacks import TensorBoard

# Define the directory for logs
logs_dir = "Logs"

# Remove the "Logs" directory if it exists
if os.path.exists(logs_dir):
    import shutil
    shutil.rmtree(logs_dir)

# Create a new "Logs" directory
os.makedirs(logs_dir)

# Create TensorBoard callback
tensorboard_callback = TensorBoard(log_dir=logs_dir)

In [5]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/50
31/31 [==============================] - 24s 740ms/step - loss: 3.2445 - accuracy: 0.0752 - val_loss: 3.2413 - val_accuracy: 0.0879
Epoch 2/50
31/31 [==============================] - 22s 713ms/step - loss: 3.2061 - accuracy: 0.0820 - val_loss: 3.2003 - val_accuracy: 0.0879
Epoch 3/50
31/31 [==============================] - 25s 824ms/step - loss: 3.1378 - accuracy: 0.0810 - val_loss: 3.0320 - val_accuracy: 0.1387
Epoch 4/50
31/31 [==============================] - 23s 753ms/step - loss: 2.9264 - accuracy: 0.0795 - val_loss: 2.7870 - val_accuracy: 0.1055
Epoch 5/50
31/31 [==============================] - 22s 722ms/step - loss: 2.7588 - accuracy: 0.1338 - val_loss: 2.5267 - val_accuracy: 0.2637
Epoch 6/50
31/31 [==============================] - 23s 745ms/step - loss: 2.3081 - accuracy: 0.2686 - val_loss: 1.8581 - val_accuracy: 0.3672
Epoch 7/50
31/31 [==============================] - 24s 757ms/step - loss: 1.8255 - accuracy: 0.3508 - val_loss: 1.3964 - val_accuracy: 0.4727

In [ ]:
%load_ext tensorboard
%tensorboard --logdir Logs

In [25]:
pip install h5py


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(validation_generator)

# Generate predictions for the validation data
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = validation_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Compute classification report
class_report = classification_report(y_true, y_pred, target_names=class_names)

# Print classification report
print("\nClassification Report:")
print(class_report)
